In [1]:
# Create a script here which takes as input the fir n cycles from a battery
# And then predicts the degredation curve 

In [145]:
import os
import sys
sys.path.insert(0,'..')
import pickle as pkl
import sys
from copy import deepcopy
from os import environ
from os.path import join as oj
import numpy as np
import torch
import torch.utils.data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import training.my_eval as my_eval
import training.data_loader as dl
from training.loss_functions import nll_loss
from training.models import *
import pandas as pd 

In [146]:
# We load the initial data, then feed it into the model so that it can 
# Predict the degredation curves for the cells 
results_path = '/Users/paolovincenzofreieslebendeblasio/Cell_Lifetime_prediction/LaurasModels/3782957491.pkl'
results = pd.Series(pkl.load(open(results_path, "rb")))

In [147]:
#Load models here 
import training.models as models
model_path  = '/Users/paolovincenzofreieslebendeblasio/Cell_Lifetime_prediction/LaurasModels'

input_dim = 1   # Number of input features (e.g. discharge capacity)
num_augment = 7  # three  values of charging schedule (avg and last) plus the variance

device = torch.device("cpu")

my_models = models.Uncertain_LSTM(1, 
                            7, 
                            num_hidden =  results.hidden_size, 
                            num_hidden_lstm =  results.hidden_size_lstm, 
                            seq_len= 100, 
                            n_layers =2, 
                            dropout =.0).to(device) 

my_models.load_state_dict(torch.load(oj(model_path,'3782957491'+".pt"),
map_location=torch.device('cpu')))
model = my_models.to(device)

capacity_output_scaler = MinMaxScaler((-1, 1), clip=False).fit(
    np.maximum(np.minimum([y[0:1]], 180), 180*0.8)
)

In [148]:
results_path = "/Users/paolovincenzofreieslebendeblasio/battery-life-prediction/"
results_filename = "14cells300CyclesData-HelgeSteinGroup.hdf5"
data_path = results_path + results_filename

#Absolute path to where the data is stored 
data_dict = dl.load_data_all_channels(data_path)
x, y, c, var = dl.get_capacity_input(
    data_dict,
    start_cycle=10,
    stop_cycle=100, 
)

qc_variance_scaler = StandardScaler().fit(var)
var = qc_variance_scaler.transform(var)
augmented_data = np.hstack([c, var])
#x = dl.scale_x(x, y)

In [149]:
max_val = 180  # nominal capacity for the cells which you are testing
end_of_life_val = (0.8 * 180)  # batteries are considered dead after 80%. This should be .8*1.1

x = np.minimum(x, max_val)
x = np.maximum(x, end_of_life_val)

x = (x - end_of_life_val) / (max_val - end_of_life_val)
print(x)

[[0.85835071 0.79935766 0.75716198 0.82812584 0.84017861 0.82845288
  0.81685328 0.80478122 0.79252653 0.78717744 0.77789454 0.77242156
  0.7585281  0.74511755 0.73680579 0.7288441  0.72434466 0.71824957
  0.72031288 0.69219746 0.6813982  0.67239602 0.66699745 0.65704984
  0.65132    0.63947746 0.63035669 0.62212549 0.61409945 0.60659723
  0.59681979 0.58963532 0.58117303 0.57057763 0.56469339 0.56076716
  0.55406906 0.53204077 0.53667535 0.52729818 0.51383755 0.50879932
  0.50077176 0.49278396 0.48451199 0.47392043 0.46349257 0.4548259
  0.44854705 0.43849283 0.43000997 0.4178245  0.40409397 0.39536797
  0.39161208 0.38745647 0.38561785 0.37407606 0.35792365 0.3463504
  0.33938449 0.33505113 0.32756052 0.32562552 0.30677927 0.29568081
  0.28912389 0.28304343 0.26911781 0.26295688 0.25535744 0.24453167
  0.23488085 0.23223112 0.21896516 0.21020675 0.20033349 0.19204543
  0.18447394 0.1705797  0.16209354 0.15626276 0.1549088  0.14475282
  0.14013738 0.1261865  0.11485898 0.10744535 0.10

In [153]:
 #Scaling the QC, and cycle life 

#x_preprocessed = dl.remove_outliers(x_scaled,y) #
x_smoothed = x

# for seq_length in [40,]:
max_steps = 1000
num_models = 1
num_samples = 20
#used_idxs = test_idxs
used_idxs = [0]
mean_val = []
cycle_dict = {}
for seq_length in [100,]:  
    test_seq_list = []
    test_life_pred_list = []
    test_seq_std_list = []
    all_outputs_arr = np.empty( (len(used_idxs),max_steps,num_models, num_samples,))
    # used_idxs =  np.arange(len(x))#for actually new data, use test_idxs=
    supp_val_data = np.hstack([c[used_idxs, :3], var[used_idxs],np.ones((len(used_idxs),1))*np.log(seq_length) ])

    #Take the scaled values for y, and the capacitance as a function of time 
    test_seq = x_preprocessed[used_idxs][:, :seq_length,None  ].copy()
    extended_seq = np.swapaxes(np.reshape(np.repeat(np.swapaxes(test_seq, 0,-1)[:,:,:,None],
                                                    num_samples, axis =-1), (1, seq_length, -1)),0,-1)

    extended_supp_data = np.swapaxes(np.reshape(np.repeat(np.swapaxes(supp_val_data, 0,-1)[:,:,None],
                                                          num_samples, axis =-1), (supp_val_data.shape[1], -1)),0,-1)

    with torch.no_grad():
        while((np.all(extended_seq[:,-1] < 1e-3) == False ) *(extended_seq.shape[1] < max_steps)):

            supp_val_data_torch = torch.from_numpy(extended_supp_data).to(device).float()

            test_seq_torch = torch.from_numpy(extended_seq[:, - seq_length:]).to(device).float()

            model.reset_hidden_state()        
            (state_mean_mean, state_var) = model(test_seq_torch, supp_val_data_torch)
            
            #Create a vector which creates the gaussian distribution
            #The number of samples, corresponds to how well we sample the gaussian space
            
            if num_samples >1:
                state_mean_noisy   = state_mean_mean  +  torch.normal(0, (torch.sqrt(state_var)))   
            else:
                state_mean_noisy   = state_mean_mean 

            state_mean_transformed = torch.from_numpy(capacity_output_scaler.inverse_transform(
                state_mean_noisy.cpu().numpy())).to(device)
            print(np.shape(test_seq_torch))
            #Inversely transforms the predicted capacity into the correct value

            mean_val.append(np.mean(state_mean_transformed.cpu().numpy()[0]))
            state_mean_transformed[:,0] = state_mean_transformed[:,0]*(test_seq_torch[:, -1, 0 ])

            extended_supp_data[:,-1] = np.log(np.exp(extended_supp_data[:,-1])+1)
            extended_seq = np.hstack([extended_seq, state_mean_transformed.cpu().numpy()[:, None]])
            #We append more and more of the extended seq state
    used_steps = extended_seq.shape[1]
    reshaped = np.swapaxes(np.reshape(np.swapaxes(extended_seq,0,1),(1,used_steps, -1, num_samples)),0,-2)
    all_outputs_arr[:,:used_steps,0,:] = reshaped[:,:,0]

all_outputs_arr = np.reshape(np.transpose(all_outputs_arr, (0,2,3, 1)), (len(used_idxs), -1, max_steps))
cycle_dict[seq_length] = np.copy(all_outputs_arr)


torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])


torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])


torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])
torch.Size([20, 100, 1])


In [152]:
import matplotlib.pyplot as plt 
plt.plot(cycle_dict[50][0][0][0:52])

KeyError: 50